In [24]:
import requests
import pandas as pd
import json

In [25]:
out = requests.get('https://covidtracking.com/api/states')
df_out = pd.DataFrame(out.json())
df_out.set_index('state', drop=True, inplace=True)

In [26]:
df_out.to_parquet('states_today.parquet')

In [27]:
ts = requests.get('https://covidtracking.com/api/states/daily')
df_ts = pd.DataFrame(ts.json())

In [28]:
i = df_ts[(df_ts['date'] == 20200313) & (df_ts.state == 'TX')].index[0]
df_ts.at[i, 'total'] = 220
df_ts.at[i, 'death'] = 0
df_ts.at[i, 'negative'] = df_ts.at[i, 'total'] - df_ts.at[i, 'positive'] 
df_ts.loc[i]

date                       20200313
state                            TX
positive                         39
negative                        181
pending                         NaN
death                             0
total                           220
dateChecked    2020-03-13T20:00:00Z
Name: 425, dtype: object

In [30]:
df_ts.to_parquet('states_daily.parquet')